## Домашняя работа
ФИО: Зырянова Татьяна Евгеньевна

## Задание 1
Реализуйте метакласс ThreadSafeSingleton, который обеспечивает создание только одного экземпляра класса, даже в многопоточной среде.
Используйте `from threading import Lock`


In [1]:
from threading import Lock

class ThreadSafeSingleton(type):
    _instances = {}
    _lock = Lock()

    def __call__(cls, *args, **kwargs):
        with cls._lock:
            if cls not in cls._instances:
                cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]

class DatabasePool(metaclass=ThreadSafeSingleton):
    def __init__(self):
        self._issued = 0

    def get_connection(self):
        self._issued += 1
        return object()


### проверка задания 1

In [2]:

# Создайте 3 экземпляра DatabasePool
pool1 = DatabasePool()
pool2 = DatabasePool()
pool3 = DatabasePool()

# Убедитесь, что это один и тот же объект
assert pool1 is pool2 is pool3

# Проверьте, что соединения разделяются между экземплярами
conn1 = pool1.get_connection()
conn2 = pool2.get_connection()
assert conn1 != conn2


## Задание 2

Создайте метакласс, который считает, сколько раз создавался каждый класс.

Требования:
1. Метакласс должен иметь атрибут _counters
1. При создании экземпляра класса счетчик должен увеличиваться
1. Добавьте метод get_count(), который возвращает количество созданных экземпляров

In [3]:
class InstanceCounter(type):
    def __call__(cls, *args, **kwargs):
        obj = super().__call__(*args, **kwargs)
        cls._count = getattr(cls, "_count", 0) + 1
        return obj

    def get_count(cls):
        return getattr(cls, "_count", 0)

class User(metaclass=InstanceCounter):
    def __init__(self, name): self.name = name

class Product(metaclass=InstanceCounter):
    def __init__(self, name): self.name = name

### проверка задания 2

In [4]:
# Проверка
user1 = User("Alice")
user2 = User("Bob")
product1 = Product("Laptop")

print(User.get_count())    # Должно быть 2
print(Product.get_count()) # Должно быть 1

2
1


## Задание 3

Создайте метакласс, который автоматически добавляет метод describe() в каждый класс.

Требования:
1. Метод describe() должен возвращать строку с именем класса
1. Используйте метакласс для создания классов Car и Book

In [5]:
class DescribeMeta(type):
    def __new__(mcls, name, bases, ns):
        if "describe" not in ns:
            def describe(self):
                return f"Это объект класса {self.__class__.__name__}"
            ns["describe"] = describe
        return super().__new__(mcls, name, bases, ns)

class Car(metaclass=DescribeMeta):
    def __init__(self, brand): self.brand = brand

class Book(metaclass=DescribeMeta):
    def __init__(self, title): self.title = title

### проверка задания 3

In [6]:
car = Car("Toyota")
book = Book("Python для начинающих")

print(car.describe())  # Должно быть "Это объект класса Car"
print(book.describe()) # Должно быть "Это объект класса Book"

Это объект класса Car
Это объект класса Book


## Задание 4

Создайте метакласс, который проверяет, что у класса есть метод save(). Можно использовать `__new__`

Требования:
1. Если у класса нет метода save(), метакласс должен выдать ошибку
1. Создайте класс User с методом save()
1. Попробуйте создать класс Message без метода save() (должна быть ошибка)

In [7]:
class SaveMeta(type):
    def __new__(cls, name, bases, attrs):
        if 'save' not in attrs:
            raise TypeError(f"Класс {name} должен реализовать метод save()")
        return super().__new__(cls, name, bases, attrs)

In [11]:
class User(metaclass=SaveMeta):
    def __init__(self, name):
        self.name = name

    def save(self):
        print(f"Пользователь {self.name} сохранён")

### проверка задания 4

In [12]:
# Проверка
user = User("Alice")
user.save()  # Должно работать

# Этот код должен вызвать ошибку:
# class Message(metaclass=SaveMeta):
#     def __init__(self, text):
#         self.text = text

Пользователь Alice сохранён


In [15]:
# Этот код должен вызвать ошибку:
class Message(metaclass=SaveMeta):
    def __init__(self, text):
        self.text = text

TypeError: Класс Message должен реализовать метод save()